## Datalake

In [17]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd


In [18]:
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
s3 = boto3.Session().client(service_name="s3", region_name=region)

role = sagemaker.get_execution_role()
account_id = boto3.client("sts").get_caller_identity().get("Account")
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [19]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-429874041769


In [20]:
# Verifying the Bucket Creation
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': '51K440M1TS79R5E4', 'HostId': '97acj8vaDyQw4DARwVFGHr9vhsm3aSjZj6JxyRvKtT1N2CRI0nTtVfeRke571YmZ0e5QpUPFXiA=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '97acj8vaDyQw4DARwVFGHr9vhsm3aSjZj6JxyRvKtT1N2CRI0nTtVfeRke571YmZ0e5QpUPFXiA=', 'x-amz-request-id': '51K440M1TS79R5E4', 'date': 'Wed, 21 May 2025 11:06:35 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [21]:
s3_private_path_maintx = "s3://{}/maintx/csv".format(bucket)
print(s3_private_path_maintx)

s3://sagemaker-us-east-1-429874041769/maintx/csv


In [22]:
!aws s3 cp "vehicle_maintenance_data.csv" $s3_private_path_maintx/

upload: ./vehicle_maintenance_data.csv to s3://sagemaker-us-east-1-429874041769/maintx/csv/vehicle_maintenance_data.csv


In [23]:
print(s3_private_path_maintx)
!aws s3 ls $s3_private_path_maintx/

s3://sagemaker-us-east-1-429874041769/maintx/csv
2025-05-21 11:06:36    5828965 vehicle_maintenance_data.csv


In [24]:
database_name = "dsomaintx"
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [25]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsomaintx


/tmp/ipykernel_2625/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [26]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2625/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsomaintx


In [27]:
table_name_csv = "maintx_raw_data"

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    Vehicle_Model string,
    Mileage int,
    Maintenance_History string,
    Reported_Issues int,
    Vehicle_Age int,
    Fuel_Type string,
    Transmission_Type string,
    Engine_Size double,
    Odometer_Reading int,
    Last_Service_Date string,
    Warranty_Expiry_Date string,
    Owner_Type string,
    Insurance_Premium int,
    Service_History int,
    Accident_History int,
    Fuel_Efficiency double,
    Tire_Condition string,
    Brake_Condition string,
    Battery_Status string,
    Need_Maintenance int
) ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_maintx
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS dsomaintx.maintx_raw_data(
    Vehicle_Model string,
    Mileage int,
    Maintenance_History string,
    Reported_Issues int,
    Vehicle_Age int,
    Fuel_Type string,
    Transmission_Type string,
    Engine_Size double,
    Odometer_Reading int,
    Last_Service_Date string,
    Warranty_Expiry_Date string,
    Owner_Type string,
    Insurance_Premium int,
    Service_History int,
    Accident_History int,
    Fuel_Efficiency double,
    Tire_Condition string,
    Brake_Condition string,
    Battery_Status string,
    Need_Maintenance int
) ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
LOCATION 's3://sagemaker-us-east-1-429874041769/maintx/csv'
TBLPROPERTIES ('skip.header.line.count'='1')


In [28]:
pd.read_sql(statement, conn)

/tmp/ipykernel_2625/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [29]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2625/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,maintx_raw_data


In [30]:
statement = """SELECT * FROM {}.{}""".format(
    database_name, table_name_csv
)

df = pd.read_sql(statement, conn)

/tmp/ipykernel_2625/2896056408.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


In [31]:
df.head(100)

,vehicle_model,mileage,maintenance_history,reported_issues,vehicle_age,fuel_type,transmission_type,engine_size,odometer_reading,last_service_date,warranty_expiry_date,owner_type,insurance_premium,service_history,accident_history,fuel_efficiency,tire_condition,brake_condition,battery_status,need_maintenance
0,Truck,58765,Good,0,4,Electric,Automatic,2000.0,28524,11/23/2023,6/24/2025,Second,20782,6,3,13.622204,New,New,Weak,1
1,Van,60353,Average,1,7,Electric,Automatic,2500.0,133630,9/21/2023,6/4/2025,Second,23489,7,0,13.625307,New,New,Weak,1
2,Bus,68072,Poor,0,2,Electric,Automatic,1500.0,34022,6/27/2023,4/27/2025,First,17979,7,0,14.306302,New,Good,Weak,1
3,Bus,60849,Average,4,5,Petrol,Automatic,2500.0,81636,8/24/2023,11/5/2025,Second,6220,7,3,18.709467,New,Worn Out,New,1
4,Bus,45742,Poor,5,1,Petrol,Manual,2000.0,97162,5/25/2023,9/14/2025,Third,16446,6,2,16.977483,Good,Good,Weak,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Motorcycle,31968,Good,0,1,Diesel,Manual,1000.0,35299,5/22/2023,3/31/2025,Second,28709,4,2,10.894891,Worn Out,Good,Weak,1
96,Van,74433,Average,4,10,Electric,Automatic,1500.0,145679,4/1/2023,1/18/2026,Second,28275,1,2,19.213680,New,New,Good,1
97,Car,45688,Poor,5,9,Electric,Manual,800.0,9054,10/19/2023,12/8/2024,First,10501,9,1,18.040922,Good,Worn Out,New,1
98,Car,78209,Average,0,4,Diesel,Manual,2500.0,45994,11/24/2023,3/24/2025,First,10483,3,2,14.296703,New,New,New,0


## EDA